In [1]:
from pynq import Overlay
from pynq import PL

OL = Overlay("/home/xilinx/pynq/overlays/WholeNetwork/design_1_wrapper.bit")
OL.ip_dict

{'axi_dma_in': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb024d530>,
  'driver': pynq.lib.dma.DMA,
  'fullpath': 'axi_dma_in',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI_LITE',
  'parameters': {'C_BASEADDR': '0x40400000',
   'C_DLYTMR_RESOLUTION': '125',
   'C_ENABLE_MULTI_CHANNEL': '0',
   'C_FAMILY': 'zynq',
   'C_HIGHADDR': '0x4040FFFF',
   'C_INCLUDE_MM2S': '1',
   'C_INCLUDE_MM2S_DRE': '1',
   'C_INCLUDE_MM2S_SF': '1',
   'C_INCLUDE_S2MM': '0',
   'C_INCLUDE_S2MM_DRE': '0',
   'C_INCLUDE_S2MM_SF': '1',
   'C_INCLUDE_SG': '0',
   'C_INCREASE_THROUGHPUT': '0',
   'C_MICRO_DMA': '0',
   'C_MM2S_BURST_SIZE': '16',
   'C_M_AXIS_MM2S_CNTRL_TDATA_WIDTH': '32',
   'C_M_AXIS_MM2S_TDATA_WIDTH': '32',
   'C_M_AXI_MM2S_ADDR_WIDTH': '64',
   'C_M_AXI_MM2S_DATA_WIDTH': '32',
   'C_M_AXI_S2MM_ADDR_WIDTH': '64',
   'C_M_AXI_S2MM_DATA_WIDTH': '32',
   'C_M_AXI_SG_ADDR_WIDTH': '64',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_NUM_MM2S_CHANNELS': '1',
   'C_NUM_S2

In [2]:
dma_in = OL.axi_dma_in
dma_out = OL.axi_dma_out
network = OL.network_0
help(dma_out)

Help on DMA in module pynq.lib.dma object:

class DMA(pynq.overlay.DefaultIP)
 |  Class for Interacting with the AXI Simple DMA Engine
 |  
 |  This class provides two attributes for the read and write channels.
 |  The read channel copies data from the stream into memory and
 |  the write channel copies data from memory to the output stream.
 |  Both channels have an identical API consisting of `transfer` and
 |  `wait` functions. If interrupts have been enabled and connected
 |  for the DMA engine then `wait_async` is also present.
 |  
 |  Buffers to be transferred must be a `PynqBuffer` objectedallocated
 |  through `pynq.allocate()` function either directly or indirectly. This
 |  means that Frames from the video subsystem can be transferred using
 |  this class.
 |  
 |  Attributes
 |  ----------
 |  recvchannel : _DMAChannel
 |      The stream to memory channel
 |  sendchannel : _DMAChannel
 |      The memory to stream channel
 |  
 |  Method resolution order:
 |      DMA
 |    

In [3]:
import numpy as np
import pynq
from pynq import Xlnk

xlnk = Xlnk()
matrix_in = xlnk.cma_array((1*28*28), dtype=np.int32)
matrix_out = xlnk.cma_array((1*28*28), dtype=np.int32)

for h in range(28):
    for w in range(28/2):
        matrix_in[h * 28 + w] = w

matrix_in_addr = hex(matrix_in.physical_address)
matrix_out_addr = hex(matrix_out.physical_address)
print(matrix_in_addr, matrix_out_addr) 

0x18049000 0x1804a000


In [ ]:
network.write(0x00, 1)
dma_in.sendchannel.transfer(matrix_in)
dma_out.recvchannel.transfer(matrix_out)
dma_in.sendchannel.wait()
for i in range(100):
    print("Control Signals", network.read(0x00))
    print(" IP Interrupt Status Register", network.read(0x0C))
    print(" DMA_Input", dma_in.read(0x04))
    print(" DMA_Output", dma_out.read(0x04))
    if network.read(0x00) & 0x02 == 2:
        break
dma_out.recvchannel.wait()






Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status Register 0
 DMA_Input 4098
 DMA_Output 0
Control Signals 1
 IP Interrupt Status R

## Address info  

    offset : bit   : function
    0x00 : Control Signals
             bit 0 : ap_start (Read/Write/COH)  
             bit 1 : ap_done  (Read/COR)  
             bit 2 : ap_idle  (Read)  
             bit 7 : auto_restart (Read/Write)  
             others: reserved  

    0x04 : Global Interrupt Enable Register
             bit 0 : Global Interrupt Enable(Read/Write)
             others: reserved

    0x08 : IP Interrupt Enable Register(Read/Write)
             bit 0 : Channel 0 (ap_done)
             bit 1 : Channel 1 (ap_ready)
             others: reserved

    0x0c : IP Interrupt Status Register(Read/Write)
             bit 0 : Channel 0 (ap_done)
             bit 1 : Channel 1 (ap_ready)
             others: reserved

    (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on HandShake)


In [6]:
matrix_in.freebuffer()
# dma_in.stop()

matrix_out.freebuffer()
# dma_out.stop()